# Desafio 2:

1. Crear sus propios vectores con Gensim basado en lo visto en clase con otro artista del dataset Songs.

2. Elegir términos de interés y buscar términos más similares y menos similares.

3. Realizar una reduccion de dimensionalidad a los embeddings, llevándolos a 2 dimensiones. Graficar los embeddings proyectados y seleccionar una cantidad de términos (variable MAX_WORDS) de forma tal que la visualización sea adecuada.

4. Inspeccionar el grafico y buscar pequeños grupos de palabras que puedan formarse. Interpretarlos e intentar obtener conclusiones. En lo posible, acompañar los grupos de palabras con capturas (y pegarlas en celdas de texto)

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
try:
  from gensim.models import Word2Vec
except:
  !pip install gensim
  from gensim.models import Word2Vec

## 1. Eligo el Dataset de Bob Marley:

In [25]:
# Descargar la carpeta de dataset
import os
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/bob-marley.txt', sep='/n', header=None)
df.head()

C:\Users\jonat\AppData\Local\Temp\ipykernel_12084\3034298888.py:4: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,"""Don't worry about a thing,"
1,'Cause every little thing gonna be all right.
2,"Singin': ""Don't worry about a thing,"
3,"'Cause every little thing gonna be all right!""..."
4,"Smiled with the risin' sun,"


In [26]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2218


### Preprocesamiento

In [27]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [28]:
# Demos un vistazo
sentence_tokens[:2]

[["don't", 'worry', 'about', 'a', 'thing'],
 ["'cause", 'every', 'little', 'thing', 'gonna', 'be', 'all', 'right']]

### Crear los vectores (word2vec)

In [29]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [30]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [31]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [32]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2218


In [33]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 532


### Entrenar embeddings

In [34]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 150191.28125
Loss after epoch 1: 93118.96875
Loss after epoch 2: 92445.78125
Loss after epoch 3: 92579.6875
Loss after epoch 4: 92034.1875
Loss after epoch 5: 88942.15625
Loss after epoch 6: 82685.0
Loss after epoch 7: 79544.5
Loss after epoch 8: 75712.9375
Loss after epoch 9: 72144.5625
Loss after epoch 10: 70732.5
Loss after epoch 11: 68090.6875
Loss after epoch 12: 62246.125
Loss after epoch 13: 60418.875
Loss after epoch 14: 60237.25
Loss after epoch 15: 59960.5
Loss after epoch 16: 58844.75
Loss after epoch 17: 58794.25
Loss after epoch 18: 58291.875
Loss after epoch 19: 57500.0


(218093, 359260)

## 2. Elijo términos de interés y busco términos más similares y menos similares.

In [35]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('protect', 0.8086336255073547),
 ('nobody', 0.8049777746200562),
 ('given', 0.8032715916633606),
 ("you've", 0.7994949221611023),
 ('really', 0.7939816117286682),
 ("feelin'", 0.7812800407409668),
 ('message', 0.7800705432891846),
 ('ask', 0.7799034714698792),
 ('mean', 0.763201117515564),
 ('door', 0.7456976771354675)]

### Explicación:
En este ejemplo "love" está relacionado con "protect" tal vez por la cación "Positive Vibration" que dice "Jah love, Jah love (protect us)".

Tambien "nobody" puede estar relacionado en la canción "All Day All Night" por la frase "I’m gonna give my love to nobody but you baby"

La palabra "given" por la cación Stand Alone" en la frase "And while you're takin' true love, you given the blame"

In [36]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('ganja', -0.20342041552066803),
 ('de', -0.20613743364810944),
 ("'pon", -0.21175745129585266),
 ('walk', -0.21341189742088318),
 ("'ypocrites", -0.21664226055145264),
 ('a', -0.22775109112262726),
 ('wall', -0.22777284681797028),
 ("smoke'a", -0.22913499176502228),
 ('gone', -0.23126623034477234),
 ('fly', -0.23371687531471252)]

### Explicación:

De las palabras menos similares tenemos sabemos que no tienen ninguna relación con "love" lo cual tiene sentido.

In [37]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["vibration"], topn=10)

[('positive', 0.9743010401725769),
 ("i'n'i", 0.9051048159599304),
 ('rastaman', 0.8862132430076599),
 ('everything', 0.8744524121284485),
 ('african', 0.8633019924163818),
 ('within', 0.8555716872215271),
 ('fadeout', 0.8545386791229248),
 ('him', 0.8513657450675964),
 ('blowing', 0.8508662581443787),
 ('human', 0.8499327301979065)]

### Explicación:

Las palabras mas comunes se relacion en la canción "positve vibration" en las frase: 
- "positive vibration"
- "I and I vibration, yeah"
- "rastaman vibration, yeah"


In [38]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(negative=["vibration"], topn=10)

[('there', -0.3677366375923157),
 ('sweet', -0.4011797606945038),
 ('a', -0.40555956959724426),
 ('more', -0.4084654450416565),
 ('place', -0.40855568647384644),
 ('dread', -0.4163437783718109),
 ('slogans', -0.4223599433898926),
 ('from', -0.42497164011001587),
 ('much', -0.42501604557037354),
 ('take', -0.42608049511909485)]

## 3. Realizar una reduccion de dimensionalidad a los embeddings, llevándolos a 2 dimensiones. Graficar los embeddings proyectados y seleccionar una cantidad de términos (variable MAX_WORDS) de forma tal que la visualización sea adecuada.

In [39]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [40]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=50
OFFSET=100

fig = px.scatter(x=vecs[OFFSET:MAX_WORDS+OFFSET,0], y=vecs[OFFSET:MAX_WORDS+OFFSET,1], text=labels[OFFSET:MAX_WORDS+OFFSET])
fig.show(renderer="vscode") # esto para plotly en colab, pero para vscode es mejor

- Por la canción "The Heathen" que dice "Di heathen back, yeah, pon di wall" es que las palabras "back", "heathen", "wall" están tan juntas.

- Por la canción "Hammer" que dice "I need a hammer", "I need a rammer". Esto hace que se relaciones muy juntas las palabras "need", "rammer" y "hammer"

- Por la canción "Soul rebel" que dice "I'm a rebel, soul rebel". Esto uno en la izquierda del gráfico las palabras "rebel" y "soul".